In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
((images_train, _), (_, _)) = fashion_mnist.load_data()

In [3]:
randomDim=100
images_train=(np.array(images_train).astype(np.float32) -127.5)/127.5
images_train.shape

(60000, 28, 28)

In [4]:
images_train=images_train.reshape(60000,28*28)
images_train.shape

(60000, 784)

In [16]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras import initializers

In [17]:
adam = Adam(learning_rate=0.0002, beta_1=0.5)  # using Legacy adam with custom learning rate to speed up training
generator = Sequential()
generator.add(Dense(256, input_dim=randomDim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
# generator.add(Dense(1024))
# generator.add(LeakyReLU(0.2))
generator.add(Dense(784, activation='tanh'))   #sigmoid [0,1], tanh[-1,1]
generator.compile(loss='binary_crossentropy', optimizer=adam)

In [18]:
discriminator = Sequential()
# discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
# discriminator.add(LeakyReLU(0.2))
# discriminator.add(Dropout(0.3))
discriminator.add(Dense(512, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

In [19]:
discriminator.trainable=False
ganInput=Input(shape=(randomDim,))
x=generator(ganInput)
ganOutput=discriminator(x)
gan=Model(inputs=ganInput,outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [20]:
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [21]:
def train(epochs=1,batchSize=64):
  batchCount=int(images_train.shape[0]/batchSize)
  print ('Epochs:', epochs)
  print ('Batch size:', batchSize)
  print ('Batches per epoch:', batchCount)

  for e in range(1, epochs+1):
    print ('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in range(batchCount):
      noise=np.random.normal(0,1,size=[batchSize,randomDim])
      imageBatch= images_train[np.random.randint(0,images_train.shape[0],size=batchSize)]

      #generate fake images
      generatedImages=generator.predict(noise)
      X = np.concatenate([imageBatch,generatedImages])

      yDis = np.zeros(2*batchSize)
      yDis[:batchSize] =0.9

      # train discriminator
      discriminator.trainable=True
      dloss=discriminator.train_on_batch(X,yDis)

      # train generator
      noise = np.random.normal(0, 1, size=[batchSize, randomDim])
      yGen = np.ones(batchSize)
      discriminator.trainable = False
      gloss = gan.train_on_batch(noise, yGen)

      if e == 1 or e % 5 == 0:
        plotGeneratedImages(e)






In [ ]:
train(100,128)

Epochs: 100
Batch size: 128
Batches per epoch: 468
--------------- Epoch 1 ---------------
4/4 [==============================] - 0s 4ms/step


4/4 [==============================] - 0s 3ms/step


<ipython-input-20-f3895603f385>:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=figsize)


4/4 [==============================] - 0s 4ms/step
